# About: 設定ファイルの変更--php.ini

---

Moodleコンテナの `php.ini` の設定内容を変更します。

## 概要

設定ファイルを変更する例としてMoodleコンテナの `php.ini` に設定されている `upload_max_filesize`, `post_max_size` の値を変更してみます。

![処理の流れ](images/moodle-031-01.png)

操作手順は以下のようになります。

1. VCノードに配置されている設定ファイルをNotebook環境に取得する
2. 取得したファイルのバックアップを作成する
3. Notebookの編集機能を利用して設定ファイルの変更をおこなう
4. 変更した設定ファイルをVCノードに配置する
5. 設定ファイルの変更を反映させるためにコンテナを再起動する

アプリケーションテンプレートによって構築したMoodle環境では、利用者が変更する可能性のある `php.ini` などの設定ファイルをVCノードに配置しています。VCノードに配置したファイルはコンテナ内から参照するために[bind mount](https://docs.docker.com/storage/bind-mounts/)の設定を行っています。このため、設定ファイルを編集する場合はコンテナ内のパス名とVCノードのパス名のどちらを指定するかについて留意する必要があります。ただし、このNotebookでは**コンテナ内のパス**のみを指定すればよいようになっています。

参考のため、Moodleコンテナにおける設定ファイルのコンテナ内のパスと、VCノードにおけるパスの対応関係を以下の表に示します。


<table>
  <tr>
    <th style="text-align:left;">コンテナ内のパス</th>
    <th style="text-align:left;">VCノードのパス</th>
  </tr>
  <tr>
    <td style="text-align:left;">/etc/php.ini</td>
    <td style="text-align:left;">/opt/moodle/{container name}/conf/php.ini</td>
  </tr>
  <tr>
    <td style="text-align:left;">/etc/php.d/</td>
    <td style="text-align:left;">/opt/moodle/{container name}/conf/php.d/</td>
  </tr>
  <tr>
    <td style="text-align:left;">/etc/php-zts.d/</td>
    <td style="text-align:left;">/opt/moodle/{container name}/conf/php-zts.d/</td>
  </tr>  <tr>
    <td style="text-align:left;">/etc/httpd/conf/httpd.conf</td>
    <td style="text-align:left;">/opt/moodle/{container name}/conf/httpd/conf/httpd.conf</td>
  </tr>
  <tr>
    <td style="text-align:left;">/etc/httpd/conf.d/</td>
    <td style="text-align:left;">/opt/moodle/{container name}/conf/httpd/conf.d/</td>
  </tr>
  <tr>
    <td style="text-align:left;">/etc/httpd/conf.modules.d/</td>
    <td style="text-align:left;">/opt/moodle/{container name}/conf/httpd/conf.modules.d/</td>
  </tr>

</table>

> `{container name}` にはコンテナ名が入ります

## パラメータの設定

変更対象のコンテナ名、設定ファイル名などを指定します。

### グループ名の指定

このNotebookの操作対象となるAnsibleのグループ名を設定します。

In [ ]:
# (例)
# ugroup_name = 'Moodle'

ugroup_name =

#### チェック

指定された `ugroup_name` の値が適切なものかチェックします。

`ugroup_name` に対応する `group_vars` ファイルが存在していることを確認します。

In [ ]:
from pathlib import Path
if not (Path('group_vars') / (ugroup_name + '.yml')).exists():
    raise RuntimeError(f"ERROR: not exists {ugroup_name + '.yml'}")

`ugroup_name`に対応する VCノードが実行中であり Ansible で操作できることを確認します。

In [ ]:
!ansible {ugroup_name} -m ping

### コンテナの指定

設定ファイルを変更する対象となるコンテナを指定します。

現在実行中のコンテナの一覧を確認します。

In [ ]:
!ansible {ugroup_name} -a 'chdir=/opt/moodle docker-compose ps --services'

表示されたコンテナ一覧から、対象とするコンテナ名を指定してください。

In [ ]:
# (例)
# target_container = 'moodle-0'

target_container =

### 設定ファイルの指定

変更する設定ファイルのパスを指定してください。ここではコンテナ内のパスを指定してください。

In [ ]:
target_file = '/etc/php.ini'

## 設定ファイルの編集

Moodleコンテナの設定ファイル`php.ini`をローカル環境に取得しJupyter Notebookの編集機能を用いて編集します。

![設定ファイルの取得、編集](images/moodle-031-03.png)

次のセルを実行すると、以下の手順が実行されます。

1. VCノードに配置されているMoodleコンテナの設定ファイル(`php.ini`)をローカル環境に取得する
2. 取得した設定ファイルのバックアップを作成する
3. Jupyter Notebookの編集機能を利用して設定ファイルを編集するためのリンクを表示する

In [ ]:
%run scripts/edit_conf.py
fetch_conf(ugroup_name, target_container, target_file)

上のセルの出力に表示されているリンクをクリックして設定ファイルの編集を行ってください。

この例では、以下の変更を行ってください。

1. `upload_max_filesize` の値を `2M`から `1024M` に変更する
1. `post_max_size` の値を `8M`から `1024M` に変更する

> メニューの[Edit]-[Find]で検索文字を入力することで、編集箇所に移動できます。
> また、ファイルの編集後は**必ず**、メニューの[File]-[Save]を選択してファイルの保存を行ってください。

ローカル環境に取得したファイルは、以下のパスに格納されています。

`./edit/{ugroup_name}/{YYYYMMDDHHmmssffffff}/php.ini`

`{ugroup_name}` には UnitGroup名が、`{YYYYMMDDHHmmssfffff}` にはファイルを取得したタイムスタンプが入ります。

また、バックアップファイルは以下のパスに格納されます。

`./edit/{ugroup_name}/{YYYYMMDDHHmmssffffff}/php.ini.orig`

次のセルを実行すると編集の前後での差分を確認することができます。

In [ ]:
%run scripts/edit_conf.py
show_local_conf_diff(ugroup_name, target_container, target_file)

## 編集した設定ファイルの反映

編集したファイルをVCノードに配置して、設定ファイルの変更内容をコンテナに反映させます。

![設定ファイルの反映](images/moodle-031-04.png)

### 変更前の状態を確認する

設定ファイル変更の反映前の状態を確認します。

MoodleのPHP情報の画面で、現在の `post_max_size`, `upload_max_filesize`の値を確認します。次のセルを実行すると MoodleのPHP情報を表示するためのリンクが表示されます。

In [ ]:
%run scripts/group.py
url = load_group_var(ugroup_name, 'moodle_url')
print(url + '/admin/phpinfo.php')

最大アップロードファイルサイズの設定値を確認します。

In [ ]:
print(url + '/admin/settings.php?section=sitepolicies')

### 編集内容の反映

前章で編集した設定ファイル `php.ini` を Moodleコンテナに反映します。

次のセルを実行すると、以下の手順が実行されます。

1. 編集前と編集後の設定ファイルの差分を表示する
2. 編集した設定ファイル(`php.ini`)をVCノードに配置する
3. コンテナを再起動して変更した設定ファイルの反映を行う

In [ ]:
apply_conf(ugroup_name, target_container, target_file)

### 変更後の状態を確認する

設定ファイル変更の反映後の状態を確認します。

MoodleのPHP情報の画面で、設定変更後の`post_max_size`, `upload_max_filesize`の値を確認します。次のセルを実行すると MoodleのPHP情報を表示するためのリンクが表示されます。

In [ ]:
print(url + '/admin/phpinfo.php')

最大アップロードファイルサイズの設定値が変更されたことを確認します。

In [ ]:
print(url + '/admin/settings.php?section=sitepolicies')

## 変更を取り消す

編集前の設定ファイルの状態に戻します。

![設定ファイル変更の取り消し](images/moodle-031-05.png)

次のセルを実行すると、以下の手順が実行されます。

1. 編集後と編集前の設定ファイルの差分を表示する
2. 編集前の設定ファイルをVCノードに配置する
3. コンテナを再起動して設定値の反映を行う

In [ ]:
revert_conf(ugroup_name, target_container, target_file)

MoodleのPHP情報の画面で、設定ファイルを元に戻した後の`post_max_size`, `upload_max_filesize`の値を確認します。

In [ ]:
print(url + '/admin/phpinfo.php')

最大アップロードファイルサイズの設定値が元に戻ったことを確認します。

In [ ]:
print(url + '/admin/settings.php?section=sitepolicies')